<a href="https://colab.research.google.com/github/Data-Science-and-Data-Analytics-Courses/UCSanDiegoX---Machine-Learning-Fundamentals-03-Jan-2019-audit/blob/master/Packages/Git.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clone

In [0]:
import os
from pathlib import Path

def clone(url, dest=".", branch="master", reloc=True):
  """
  Clone remote branch from url into dest
  branch not provided: clone all branches
  reloc is True: relocate to repository
  """

  url = url.strip(" /")
  repo = Path(dest, os.path.basename(url)).resolve()

  # dest must not be inside existing repository
  is_out = !git -C "$dest" rev-parse
  if not is_out: # inside repository
    raise ValueError("Can't clone into existing repository")
  
  # Clone
  p = repo.as_posix()
  if branch: # specific branch
    !git clone --single-branch --branch "$branch" "$url" "$p"
  else: # all branches
    !git clone "$url" "$p"
  
  # Relocate
  if reloc:
    os.chdir(repo)

  return repo.as_posix()
  
if __name__ == "__main__":
  clone("https://github.com/Data-Science-and-Data-Analytics-Courses/UCSanDiegoX---Machine-Learning-Fundamentals-03-Jan-2019-audit")

# Configure

In [0]:
from google.colab import files
import json

def config(configs=None, level="--global"):
  """
  Configure Git
  configs not provided: user provides .json file containing text: {"email": "<email>", "name": "<display name>", ...}
  level: -—local|--global|--system
  """

  # Configurations
  if not configs:
    uploaded = files.upload() # upload .json file
    try:
      [content] = uploaded.values() # bytestring in .json file
      configs = json.loads(content)
    finally:
      for p in uploaded:
        !rm "$p"
      print("Removed all uploaded")
  
  # Configure
  email = configs["email"]
  name = configs["name"]
  !git config "$level" user.email "$email"
  !git config "$level" user.name "$name"

# Push

In [0]:
from google.colab import files
import json
from urllib.parse import urlsplit

def push(url, branch="master", account=None):
  """
  Push branch to url
  account not provided: user provides .json file containing text: {"username": "<username>", "password": "<password>"}
  branch not provided: push all branches
  """
  
  rurl = urlsplit(url)
  
  # Account information
  if not account:
    uploaded = files.upload() # upload .json file
    try:
      [content] = uploaded.values() # bytestring in .json file
      account = json.loads(content)
    finally:
      for p in uploaded:
        !rm "$p"
      print("Removed all uploaded")
  
  # Construct authentication remote
  rauth = rurl._replace(netloc="{username}:{password}@{hostname}".format(username=account["username"], password=account["password"], hostname=rurl.hostname)) # add username and password
  
  # Push
  remote = rauth.geturl()
  if branch:
    input("Press Enter to push {}...".format(branch))
    !git push "$remote" "$branch"
  else:
    input("Press Enter to push all branches...")
    !git push --all "$remote"

# push("https://github.com/Data-Science-and-Data-Analytics-Courses/UCSanDiegoX---Machine-Learning-Fundamentals-03-Jan-2019-audit")

SyntaxError: ignored